In [16]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator
import sys
import numpy as np
import math

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, BasicAer, IBMQ
from qiskit.visualization import plot_histogram, plot_bloch_multivector

from qiskit.extensions import Initialize

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

ibmqfactory.load_account:WARNING:2022-09-08 19:47:01,403: Credentials are already in use. The existing account in the session will be replaced.


In [17]:
CABRA  = "ATGGTGCTGTCTGCCGCCGACAAGTCCAATGTCAAGGCCGCCTGGGGCAAGGTTGGCAGCAACGCTGGAGCTTATGGCGCAGAGGCTCTGGAGAGGATGTTCCTGAGCTTCCCCACCACCAAGACCTACTTCCCCCACTTCGACCTGAGCCACGGCTCGGCCCAGGTCAAGGGCCACGGCGAGAAGGTGGCCGCCGCGCTGACCAAAGCGGTGGGCCACCTGGACGACCTGCCCGGTACTCTGTCTGATCTGAGTGACCTGCACGCCCACAAGCTGCGTGTGGACCCGGTCAACTTTAAGCTTCTGAGCCACTCCCTGCTGGTGACCCTGGCCTGCCACCACCCCAGTGATTTCACCCCCGCGGTCCACGCCTCCCTGGACAAGTTCTTGGCCAACGTGAGCACCGTGCTGACCTCCAAATACCGTTAA"
CABALLO = "ATGGTGCTGTCTGCCGCCGACAAGACCAACGTCAAGGCCGCCTGGAGTAAGGTTGGCGGCCACGCTGGCGAGTATGGCGCAGAGGCCCTAGAGAGGATGTTCCTGGGCTTCCCCACCACCAAGACCTACTTCCCCCACTTCGATCTGAGCCACGGCTCCGCCCAGGTCAAGGCCCACGGCCAGAAGGTGGGCGACGCGCTGACTCTCGCCGTGGGCCACCTGGACGACCTGCCTGGCGCCCTGTCGAATCTGAGCGACCTGCACGCACACAAGCTGCGCGTGGACCCCGTCAACTTCAAGCTCCTGAGTCATTGCCTGCTGTCCACCTTGGCCGTCCACCTCCCCAACGATTTCACCCCTGCCGTCCACGCCTCCCTGGACAAGTTCTTGAGCAGTGTGAGCACCGTGCTGACCTCCAAATACCGTTAA"
PATO = "ATGGTGCTGTCTGCGGCTGACAAGACCAACGTCAAGGGTGTCTTCTCCAAAATCGGTGGCCATGCTGAGGAGTATGGCGCCGAGACCCTGGAGAGGATGTTCATCGCCTACCCCCAGACCAAGACCTACTTCCCCCACTTTGACCTGCAGCACGGCTCTGCTCAGATCAAGGCCCATGGCAAGAAGGTGGCGGCTGCCCTAGTTGAAGCTGTCAACCACATCGATGACATTGCGGGTGCTCTCTCCAAGCTCAGTGACCTCCACGCCCAAAAGCTCCGTGTGGACCCTGTCAACTTCAAATTCCTGGGCCACTGCTTCCTGGTGGTGGTTGCCATCCACCACCCCGCTGCCCTGACCCCAGAGGTCCACGCTTCCCTGGACAAGTTCATGTGCGCCGTGGGTGCTGTGCTGACTGCCAAGTACCGTTAG"

In [18]:
def encode_bitstring(bitstring, qr, cr, inverse=False):
    """
    create a circuit for constructing the quantum superposition of the bitstring
    """
    n = math.ceil(math.log2(len(bitstring))) + 2                 #number of qubits
    assert n > 4, "the length of bitstring must be at least 4"
    
    qc = QuantumCircuit(qr, cr)
    
    #the probability amplitude of the desired state
    desired_vector = np.array([ 0.0 for i in range(2**n) ])     #initialize to zero

################################################################################################################    
 #         Cambio requerido en el código para corregir error presentado   #
    
    qc_init = QuantumCircuit(n) # Creación de compuertas circuitales para inicialización
    inverse_qc_init = QuantumCircuit(n) # Creación de compuertas circuitales para inversión de la inicialización

################################################################################################################

    amplitude = np.sqrt(1.0/len(bitstring))
    
    for i, b in enumerate(bitstring):
        pos = i * 4
        if b == "A":
            desired_vector[pos] = amplitude
        elif b=="C":
            desired_vector[pos+1] = amplitude
        elif b=="G":
            desired_vector[pos+2] = amplitude
        elif b=="T":
            desired_vector[pos+3] = amplitude
        
            
        

################################################################################################################    
 #         Cambio requerido en el código para corregir error presentado   #
    
    qc_init = Initialize(desired_vector)
    
 ################################################################################################################       
    
    if not inverse:
#         qc.initialize(desired_vector, qr)

        qc.append(qc_init, qr)  #  Cambio requerido en el código para corregir error presentado   #
    
        qc.barrier(qr)
        
    else:
#         qc.initialize(desired_vector, qr).inverse()  #invert the circuit

        inverse_qc_init = qc_init.gates_to_uncompute()  # Cambio requerido en el código para corregir error presentado   #
        qc.append(inverse_qc_init, qr)  # Cambio requerido en el código para corregir error presentado   #
        
        qc.barrier(qr)
        for i in range(n):
            qc.measure(qr[i], cr[i])
    print()
    return qc

In [19]:
n = math.ceil(math.log2(len(CABALLO))) + 2               #number of qubits
qr = QuantumRegister(n)
cr = ClassicalRegister(n)

qc_cabra     = encode_bitstring(CABRA, qr, cr)
qc_cabra.draw()
qc_caballo = encode_bitstring(CABALLO, qr, cr)
qc_caballo.draw()
qc_pato = encode_bitstring(PATO, qr, cr)
qc_pato.draw()

circs = {"CABRA": qc_cabra, "CABALL0": qc_caballo, "PATO": qc_pato}

In [20]:
inverse_qc_cabra     = encode_bitstring(CABRA,     qr, cr, inverse=True)
inverse_qc_caballo = encode_bitstring(CABALLO, qr, cr, inverse=True)
inverse_qc_pato = encode_bitstring(PATO, qr, cr, inverse=True)

inverse_circs = {"CABRA": inverse_qc_cabra, "CABALLO": inverse_qc_caballo, "PATO": inverse_qc_pato}

In [21]:
from qiskit import IBMQ, BasicAer

key = "CABRA"       #the name of the code used as key to find similar ones

# use local simulator
backend = BasicAer.get_backend("qasm_simulator")
shots = 8192

combined_circs = {}
count = {}

most_similar, most_similar_score = "", -1.0

for other_key in inverse_circs:
    if other_key == key:
        continue
        
    combined_circs[other_key] = circs[key].compose(inverse_circs[other_key])   #combined circuits to look for similar codes
    job = execute(combined_circs[other_key], backend=backend,shots=shots)
    st = job.result().get_counts(combined_circs[other_key])
    if "0"*n in st:
        sim_score = st["0"*n]/shots
    else:
        sim_score = 0.0
    
    print("Similarity score of",key,"and",other_key,"is",sim_score)
    if most_similar_score < sim_score:
        most_similar, most_similar_score = other_key, sim_score

print("[ANSWER]", key,"is most similar to", most_similar)

Similarity score of CABRA and CABALLO is 0.77099609375
Similarity score of CABRA and PATO is 0.550537109375
[ANSWER] CABRA is most similar to CABALLO
